# Align images using astroalign
Notebook which aligns two (or three) images taken with the PESTO instrument AFTER running through the reduction pipeline [here](https://reductionpipeline.readthedocs.io/en/latest/).

The images will all be aligned with respect to the first filter

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import astroalign as aa
import numpy as np
import time
from astropy.wcs import WCS

In [2]:
object_name = 'NGC6207'
filter1 = 'ha'
filter2 = 'g'
filter3 = 'z'

In [3]:
NGC_1 = fits.open(object_name+'_'+filter1+'.fits')[0].data.byteswap().newbyteorder()
NGC_2 = fits.open(object_name+'_'+filter2+'.fits')[0].data.byteswap().newbyteorder()
NGC_3 = fits.open(object_name+'_'+filter3+'.fits')[0].data.byteswap().newbyteorder()

## Align
We will now realign the g and i bands to be concident with ha (or whatever bands you are using)

In [6]:
t, __ = aa.find_transform(NGC_2, NGC_1)
NGC_2_aligned, footprint = aa.apply_transform(t, NGC_2, NGC_1)

t, __ = aa.find_transform(NGC_3, NGC_1)
NGC_3_aligned, footprint = aa.apply_transform(t, NGC_3, NGC_1)

## Save
Now we save the realigned images as fits images using the ha WCS

In [7]:
hdr = WCS(fits.open(object_name+'_'+filter1+'.fits')[0].header).to_header()
hdr['Comment'] = 'Updated to have the correct alignment based off Halpha image on %s by Carter Rhea'%(str(time.time()))
fits.writeto(object_name+'_'+filter2+'_aligned.fits', NGC_2_aligned, hdr, overwrite=True)
fits.writeto(object_name+'_'+filter3+'_aligned.fits', NGC_3_aligned, hdr, overwrite=True)